In [21]:
from deepcave import Recorder, Objective
from deepcave.runs.converters.deepcave import DeepCAVERun
from deepcave.plugins.hyperparameter.importances import Importances
import pandas as pd
from arlbench.core.algorithms import DQN, PPO, SAC
from pathlib import Path
from ConfigSpace import ConfigurationSpace, Float, Categorical
import math
import os

In [22]:
replacement_dict = {"atari_qbert": "Atari", 'atari_double_dunk': "Atari", 'atari_phoenix': "Atari", 'atari_this_game': "Atari", 'atari_battle_zone': "Atari", 
                    'box2d_lunar_lander': "Box2d", "box2d_bipedal_walker": "Box2d", 'cc_acrobot': "CC", 'cc_cartpole': "CC", 'cc_mountain_car': "CC", "cc_pendulum": "CC", 'cc_continuous_mountain_car': "CC",
                    'minigrid_door_key': "XLand", 'minigrid_empty_random': "XLand", 'minigrid_four_rooms': "XLand", 
                    'minigrid_unlock': "XLand", "brax_ant": "Brax", "brax_halfcheetah": "Brax", "brax_hopper": "Brax", "brax_humanoid": "Brax"}
algorithms = {"dqn": DQN, "ppo": PPO, "sac": SAC}
envs = ["atari_qbert", 'atari_double_dunk', 'atari_phoenix', 'atari_this_game', 'atari_battle_zone', 
        'box2d_lunar_lander', 'box2d_bipedal_walker', 'cc_acrobot', 'cc_cartpole', 'cc_mountain_car', 'cc_pendulum', 'cc_continuous_mountain_car',
        'minigrid_door_key', 'minigrid_empty_random', 'minigrid_four_rooms', 'minigrid_unlock', 'brax_ant', 'brax_halfcheetah', 'brax_hopper', 'brax_humanoid']


In [23]:
def data_to_deepcave(algorithm, domain=False, save_path=None):
    target_path = f"../results_combined/sobol/{domain}_{algorithm}.csv"
    try:
        all_configs = pd.read_csv(target_path)
    except:
        print(f"Could not read {target_path}")
        return

    default_configspace = algorithms[algorithm].get_hpo_search_space().get_hyperparameter_names()
    configspace = ConfigurationSpace()
    for c in default_configspace:
        if c in ["buffer_prio_sampling", "use_target_network", "alpha_auto", "normalize_advantage"]:
            hp = Categorical(c, [True, False])
            configspace.add_hyperparameter(hp)
        else:
            key = [a for a in all_configs.keys() if c in a]
            if len(key) > 0:
                key = key[0]
                hp_min = all_configs[key].min()
                hp_max = all_configs[key].max()
                bounds = (min(0, hp_min-0.1*hp_min), hp_max+0.1*hp_max)
                hp = Float(c, bounds=bounds)
                configspace.add_hyperparameter(hp)
    default = configspace.get_default_configuration()
    accuracy = Objective("reward_mean", lower=min(all_configs["performance"]), upper=max(all_configs["performance"]), optimize="upper")
    save_path = save_path if save_path else f"deepcave_logs/{algorithm}_{domain}"

    with Recorder(configspace, objectives=[accuracy], save_path=save_path) as r:
        for index, d in all_configs.iterrows():
            current_hps = d
            config = default
            for c in current_hps.keys():
                key = c.split(".")[-1]
                if key in config.keys():
                    if math.isnan(current_hps[c]):
                        current_hps[c] = 0
                    try:
                        value = float(current_hps[c])
                    except:
                        current_hps[c]
                    config[key] = value
            r.start(config=config, budget=1, seed=d["seed"])
            r.end(config=config, costs=current_hps["performance"], budget=1, seed=d["seed"])
    return save_path

In [24]:
def get_importances(path, algorithm, method="local"):
    # Instantiate the run
    run = DeepCAVERun.from_path(Path(path))
    objective_id = run.get_objective_ids()[0]
    budget_ids = run.get_budget_ids()

    # Instantiate the plugin
    plugin = Importances()

    inputs = plugin.generate_inputs(
        objective_id=objective_id,
        hyperparameter_names= algorithms[algorithm].get_hpo_search_space().get_hyperparameter_names(), 
        budget_ids=budget_ids,
        method=method, n_hps=10, n_trees=10
    )
    # Note: Filter variables are not considered.
    outputs = plugin.generate_outputs(run, inputs)

    # Finally, you can load the figure. Here, the filter variables play a role.
    # Alternatively: Use the matplotlib output (`load_mpl_outputs`) if available.
    figure = plugin.load_outputs(run, inputs, outputs)  # plotly.go figure
    return figure


def get_importances_values(path, algorithm, method="local", threshold=0.05):
    from deepcave.evaluators.fanova import fANOVA
    
    # Instantiate the run
    run = DeepCAVERun.from_path(Path(path))

    # Instantiate the plugin
    fanova = fANOVA(run=run)

    fanova.calculate(n_trees=10)

    importances = fanova.get_importances()

    importances_over_threshold = {}

    for hp, importance in importances.items():
        importance = importance[0]  # we use the mean

        if importance >= threshold:
            importances_over_threshold[hp] = importance    

    return importances_over_threshold

In [25]:
algorithm = "dqn"
deepcave_paths = []
for env in envs:
   if not os.path.isdir(f"deepcave_logs/{algorithm}_{env}"):
       print(f"{env} not found")

for env in envs:
   if not os.path.isdir(f"deepcave_logs/{algorithm}_{env}"):
       data_path = data_to_deepcave(algorithm, domain=env)
       deepcave_paths.append(data_path)

box2d_bipedal_walker not found
cc_acrobot not found
cc_cartpole not found
cc_pendulum not found
cc_continuous_mountain_car not found
brax_ant not found
brax_halfcheetah not found
brax_hopper not found
brax_humanoid not found
Could not read ../results_combined/sobol/box2d_bipedal_walker_dqn.csv
Could not read ../results_combined/sobol/cc_pendulum_dqn.csv
Could not read ../results_combined/sobol/cc_continuous_mountain_car_dqn.csv
Could not read ../results_combined/sobol/brax_ant_dqn.csv
Could not read ../results_combined/sobol/brax_halfcheetah_dqn.csv
Could not read ../results_combined/sobol/brax_hopper_dqn.csv
Could not read ../results_combined/sobol/brax_humanoid_dqn.csv


In [26]:
from collections import defaultdict
import numpy as np


SUBSETS = {
    "ppo": ["box2d_lunar_lander", "brax_humanoid", "atari_battle_zone", "atari_phoenix", "minigrid_empty_random"],
    "dqn": ["cc_acrobot", "atari_this_game", "atari_double_dunk", "minigrid_four_rooms", "minigird_empty_random"],
    "sac": ["box2d_bipedal_walker", "brax_halfcheetah", "brax_hopper", "cc_continuous_mountain_car"]
}


CATEGORIES = {
    "ppo": ["Atari", "Box2D", "CC", "XLand", "Brax"],
    "dqn": ["Atari", "Box2D", "CC", "XLand"],
    "sac": ["Box2D", "CC", "Brax"]
}


paths = []
for env in envs:
   if os.path.isdir(f"deepcave_logs/{algorithm}_{env}"):
       paths.append(f"deepcave_logs/{algorithm}_{env}")
for data_path in paths:
    if not os.path.isfile(data_path + "/importances_global.png"):
        print(f"Importances for {data_path} not found")


# find HPs with importance > 10%, 5%, 3%
for threshold in [0.1, 0.05, 0.03]:
    print(f"### Threshold = {threshold} ###")
    importances_per_category = defaultdict(list)
    all_importances = []
    subset_importances = []
    
    for data_path in paths:
        experiment = data_path.split('/')[1]
        algorithm = experiment[:3]
        exp_name = experiment[4:]
        
        importances_over_threshold = get_importances_values(Path(data_path) / "run_1", algorithm, method="global", threshold=threshold)

        
        count = len(importances_over_threshold)
        category = replacement_dict[exp_name]

        print(f"Experiment: {experiment} ({category}) = {len(importances_over_threshold)}")
        
        importances_per_category[category] += [count]
        all_importances += [count]

        if exp_name in SUBSETS[algorithm]:
            subset_importances += [count]

    string = f"HPs with over ${int(100 * threshold)}\%$ importance "
    cat = CATEGORIES[algorithm]

    for c in cat:
        string += f"& {np.mean(importances_per_category[c]):.2f} "

    string += f"& {np.mean(all_importances):.2f}"
    string += f"& {np.mean(subset_importances):.2f}"
    print(string)

for data_path in paths:
    if not os.path.isfile(data_path + "/importances_global.png"):
        #try:
        img = get_importances(Path(data_path) / "run_1", algorithm, method="global")
        img.write_image(data_path + "/importances_global.png")
        #except Exception as e:
        #    print("Error")
        #    print(e)

Importances for deepcave_logs/dqn_cc_acrobot not found
Importances for deepcave_logs/dqn_cc_cartpole not found
22.53125 432.94531
20.703120000000013 432.94531
17.09375 432.94531
21.703120000000013 432.94531
28.601560000000006 432.94531
22.210935000000006 432.94531
17.671870000000013 432.94531
24.6875 432.94531
20.34375 432.94531
13.46875 432.94531
432.94531 432.94531
432.94531 432.94531
432.94531 432.94531
162.01562 432.94531
345.74219 432.94531
432.94531 432.94531
426.70311 432.94531
432.94531 432.94531
205.03906 432.94531
432.94531 432.94531
20.8125 432.94531
17.914060000000006 432.94531
432.94531 432.94531
25.898435000000006 432.94531
25.921870000000013 432.94531
24.414060000000006 432.94531
323.01561 432.94531
22.828120000000013 432.94531
393.94531 432.94531
23.515620000000013 432.94531
17.945310000000006 432.94531
29.40625 432.94531
12.695310000000006 432.94531
24.25 432.94531
16.257810000000006 432.94531
17.765620000000013 432.94531
17.992185000000006 432.94531
29.398435000000006

In [27]:
for data_path in paths:
    if not os.path.isfile(data_path + "/importances_global.png"):
        print(f"Importances for {data_path} not found")